In [ ]:
!pip install tabulate
!pip install pyathena

In [ ]:
import pandas as pd
import numpy as np
import pickle
from pyathena import connect
from pyathena.pandas.cursor import PandasCursor

In [ ]:
class TrafficDataDC:
        
    def run_query(self, date=None, limit=0, table_name = 'uwdatascience2023.full_harddrivetraffic'):
        self.query = f"select * from {table_name} "
        if date:
            self.query += f"WHERE month_end = date('{date}') "
        if limit != 0:
            self.query += "limit "+str(limit)
        print("Executing \n", self.query)
        
        self.run_athena_query(print_out=False)

    def run_athena_query(self, print_out=False):
        cursor = connect(
            region_name='us-west-2',
            work_group="primary",
            cursor_class=PandasCursor).cursor()

        self.df = cursor.execute(self.query).as_pandas()

        if print_out:
            print(self.df.to_markdown(index=False))

    def fill_data(self):
        self.df = self.df.sort_values(by=['chunk_id', 'timestamp_nano'])
        self.df['container_group'] = self.df.groupby('chunk_id')['container_group'].ffill()
        self.df['container_encoding'] = self.df.groupby('chunk_id')['container_encoding'].ffill()
        self.df['chunk_size'] = self.df.groupby('chunk_id')['chunk_size'].ffill()
        self.df.dropna(inplace=True)

    def convert_date(self, col = 'timestamp_nano', unit = 'ns', new_col = 'datetime'):
        self.df[new_col] = pd.to_datetime(self.df[col], unit=unit)
        self.drop_col(cols=[col])

    def drop_col(self, cols = []):
        self.df.drop(columns=cols, inplace=True)

    def convert_chunk_int(self, col = 'chunk_size'):
        self.df[col] = self.df[col].astype(int)
        
    def filter_rows(self, date):
        cutoff_date = pd.to_datetime(date)
        # Filter rows where the date is less than or equal to the cutoff_date
        self.df = self.df[self.df['datetime'] >= cutoff_date]
        
    def process(self):
        self.fill_data()
        self.convert_date()
        self.drop_col()
        self.filter_rows('2021-01-01')
        self.convert_chunk_int()
        
    def save_model(self, filename='TrafficDataDC.pickle'):
        with open(filename, 'wb') as f:
            pickle.dump(self, f)

In [ ]:
s3_traffic = TrafficDataDC()
s3_traffic.run_query(date="2022-01-31", limit = 300000)

In [ ]:
s3_traffic.process()

In [ ]:
print(len(s3_traffic.df))

In [ ]:
s3_traffic.df

In [ ]:
s3_traffic.save_model()